# Removing Dateless items

In [1]:
import pandas as pd
import datetime
import itertools
import numpy as np

In [2]:
dates =pd.read_table("cit-HepPh-dates.txt", header = None)
dates.columns =['id', 'date']

In [3]:
raw = pd.read_table("Cit-HepPh.txt", header = None)

In [4]:
dataset =set(raw.iloc[:,1].tolist()+raw.iloc[:,0].tolist())

In [5]:
dateset =set(dates.iloc[:,0])

In [6]:
uniq = list(dataset & dateset)

In [7]:
isuniq = raw.isin(uniq)

In [8]:
data = raw[isuniq.iloc[:,1] & isuniq.iloc[:,0]]
data.columns =['id1', 'id2']

In [9]:
data.shape

(347268, 2)

In [10]:
raw.shape

(421578, 2)

In [11]:
data = data.merge(dates, left_on="id1", right_on = "id")

In [12]:
data = data.merge(dates, left_on="id2", right_on="id")

In [13]:
del data['id_x']
del data['id_y']

In [14]:
data['date_x'] = pd.to_datetime(data['date_x'])
data['date_y'] = pd.to_datetime(data['date_y'])

data  = data.iloc[data['date_x'].argsort(),:]

In [15]:
data = data[data.date_x>data.date_y]

In [16]:
data = data.reset_index()
del data['index']

In [17]:
data.shape

(347078, 4)

# Sample Creation

trainingsize= 10000
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[0:i]))==trainingsize:
        print(i)
        train_end = i
        break
        
validationsize = 100
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[train_end:i]))==validationsize:
        print(i)
        val_end = i
        break


testsize = 100
for i in range(data.shape[0]):
    if len(set(data['id1'].loc[val_end:i]))==validationsize:
        print(i)
        test_end = i
        break
        
        
#manual check, change  train end to 75052(-3)
#manual check, change val end to 76192(+48)
# change test end to  77395(+50)

In [18]:
train_end = 75246
val_end = 76228
test_end = 77519

train_end_new = 75246
val_end_new = 76228
test_end_new = 77519


In [19]:
train_end_new = 75246

In [20]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [21]:
train = data.loc[0:train_end_new]

In [22]:
node_dict = {}
node = 1
for id1 in f7(train.id1):
    for id2 in train[train.id1 ==id1].id2:
        if id2 in node_dict:
            continue
        else:
            node_dict[id2] = node
            node +=1
    if id1 in node_dict:
        continue
    else:
        node_dict[id1] = node
        node +=1
            

In [23]:
inv_map = {v: k for k, v in node_dict.items()}

In [24]:
d = pd.Series(node_dict, name = "node")
d.index.name = "id"
d = d.reset_index()

In [25]:
train =train.merge(d, left_on ='id1', right_on = 'id')
train =train.merge(d, left_on ='id2', right_on = 'id')
del train['id_x']
del train['id_y']
train = train.sort_values(['date_x'])

In [26]:
train.tail()

,id1,id2,date_x,date_y,node_x,node_y
74648,9706488,9701230,1997-06-24,1997-01-06,11676,10144
16835,9706484,9307291,1997-06-24,1993-07-18,11678,695
47453,9706496,9503484,1997-06-24,1995-03-31,11680,4548
57468,9706486,9511411,1997-06-24,1995-11-24,11666,6495
75246,9706487,9702314,1997-06-24,1997-02-12,11679,10453


train.to_csv('training.csv',index  = False)

In [ ]:
for i in   [2351, 4459,  7051,  8251,  9215,  9238,  9485,  9890,  9965, 11089]:
    article = inv_map[i]
    print(i)
    print(len(train[train.id1 == article])+len(train[train.id2 == article]))
    print(len(data[data.id1 == article])+len(data[data.id2 == article]))


In [115]:
val_end_new = 75246+1269
val = data.loc[0:val_end_new]


Find id2 values which were not present in training set

In [116]:
unwanted = list(set(val.id2) - set(train.id2) )

In [117]:
val = val.loc[[True if i not in unwanted else False for i in val.id2]]

In [118]:
node_dict = {}
node = 1
for id1 in f7(val.id1):
    for id2 in val[val.id1 ==id1].id2:
        if id2 in node_dict:
            continue
        else:
            node_dict[id2] = node
            node +=1
    if id1 in node_dict:
        continue
    else:
        node_dict[id1] = node
        node +=1
            

In [119]:
inv_map = {v: k for k, v in node_dict.items()}

In [120]:
d = pd.Series(node_dict, name = "node")
d.index.name = "id"
d = d.reset_index()

In [121]:
val =val.merge(d, left_on ='id1', right_on = 'id')
val =val.merge(d, left_on ='id2', right_on = 'id')
del val['id_x']
del val['id_y']
val = val.sort_values(['date_x'])

In [122]:
len(set(val.id1)-set(train.id1))

99

In [123]:
val.to_csv('validation.csv',index  = False)